In [3]:
!python PLT_BFA_env_labelling.py

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import glob

In [23]:
input_files = glob.glob('../BFA_data/hBFA1*/*_bc_counts_clustered.csv')
print(input_files)

['../BFA_data/hBFA1_part1/hBFA1_part1_bc_counts_clustered.csv', '../BFA_data/hBFA1_part2/hBFA1_part2_bc_counts_clustered.csv']


In [24]:
dats = []
for f in input_files:
    td = pd.read_csv(f)
    td['Full.BC'] = td['Diverse.BC'] + td['Environment.BC']
    dats.append(td)

In [25]:
for i in range(len(dats)):
    for j in range(i+1, len(dats)):
        print(input_files[i].split('/')[-1], input_files[j].split('/')[-1], len(set(dats[i]['Full.BC']).intersection(set(dats[j]['Full.BC']))))

hBFA1_part1_bc_counts_clustered.csv hBFA1_part2_bc_counts_clustered.csv 2664


In [26]:
merge_type = 'inner'
bc_cols = ['Full.BC', 'Diverse.BC', 'Environment.BC']
time_cols = [c for c in dats[0].columns if 'Time' in c]
running_dat = dats[0][bc_cols + time_cols]
for d in dats[1:]:
    tmp_time_cols = [c for c in d.columns if 'Time' in c]
    repeat_time_cols = [c for c in tmp_time_cols if c in time_cols]
    new_time_cols = [c for c in tmp_time_cols if c not in time_cols]
    running_dat = running_dat.merge(d, on=bc_cols, how=merge_type, suffixes=('_old1', '_new1')).fillna(0)
    for t in repeat_time_cols:
        running_dat[t] = running_dat[t + '_old1'] + running_dat[t + '_new1']
    time_cols += new_time_cols
    running_dat = running_dat[bc_cols + time_cols]

In [27]:
running_dat['Total.Counts'] = running_dat.apply(lambda drow: sum([drow[i] for i in time_cols]), axis=1)
use_cols = bc_cols + ['Total.Counts'] + sorted(time_cols,
                                               key=lambda c: c[:c.index('Time')+4] + c[c.index('Time')+4:].zfill(3))
# excluding chimeras
dd = dict(running_dat['Diverse.BC'].value_counts())
dbc_with_mult_hits = set([d for d in running_dat['Diverse.BC'] if dd[d] > 1])
fullbc_chimeras = []
for dbc in dbc_with_mult_hits:
    td2 = running_dat.loc[running_dat['Diverse.BC'] == dbc]
    max_tot_reads = max(td2['Total.Counts'])
    for entry in td2.as_matrix(['Full.BC', 'Total.Counts']):
        if entry[1]/max_tot_reads < 0.01:
            fullbc_chimeras.append(entry[0])
running_dat['is_chimeric'] = running_dat.apply(lambda drow: drow['Full.BC'] in fullbc_chimeras, axis=1)
d_no_chimeras = running_dat.loc[~running_dat['is_chimeric']]
d_yes_chimeras = running_dat.loc[running_dat['is_chimeric']]

In [28]:
print(len(d_no_chimeras), len(d_yes_chimeras))

2587 77
